### Model Training

Once your model is defined, it can be trained. This can happen on a single thread or on a parallel framework like Watson Machine Learning or Apache Spark. In the most simple case Model Definition and Model Training is just a couple of LOCs (lines of code) away. In the case of Watson Machine Learning or Apache Spark models might need to get serialized and transferred to another technology / framework.

Please specify and justify the technologies used for model definition and training in the ADD. 

Once you think you have achieved a descent model performance save the notebook according to the process model’s naming convention and proceed to the model evaluation task.

In [1]:
!pip install tensorflow
!pip install python-mnist
!pip install Pillow
!pip install pyspark

Waiting for a Spark session to start...
Spark Initialization Done! ApplicationId = app-20210611174638-0000
KERNEL_ID = f43bf63d-ddba-486f-a765-e76db0a6234c
     |████████████████████████████████| 454.3 MB 27 kB/s s eta 0:00:013
     |████████████████████████████████| 1.2 MB 68.5 MB/s eta 0:00:01
     |████████████████████████████████| 1.0 MB 71.6 MB/s eta 0:00:01
     |████████████████████████████████| 65 kB 3.9 MB/s  eta 0:00:01
     |████████████████████████████████| 462 kB 82.1 MB/s eta 0:00:01
     |████████████████████████████████| 14.8 MB 77.1 MB/s eta 0:00:01
     |████████████████████████████████| 4.0 MB 75.5 MB/s eta 0:00:01
     |████████████████████████████████| 129 kB 83.6 MB/s eta 0:00:01
     |████████████████████████████████| 57 kB 3.9 MB/s  eta 0:00:01
     |████████████████████████████████| 6.0 MB 85.6 MB/s eta 0:00:01
     |████████████████████████████████| 4.0 MB 57.5 MB/s eta 0:00:01
     |████████████████████████████████| 42 kB 1.4 MB/s  eta 0:00:01
     |█████████

In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib import image
import tensorflow as tf
import seaborn as sns
from mnist import MNIST
import numpy as np
import PIL
from PIL import Image
import os
import matplotlib.image as mping
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
from pyspark.ml.feature import StringIndexer
from numpy import asarray
import pandas as pd
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout
from tensorflow.keras import backend as K
from sklearn.neighbors import KNeighborsClassifier
from sklearn.decomposition import PCA

tf.__version__

'2.5.0'

In [3]:
PIL.__version__

'8.2.0'

In [4]:
# fire up the spark session
# comment this cell out for a spark server
#sc = SparkContext.getOrCreate(SparkConf().setMaster("local[*]"))

#spark = SparkSession \
#    .builder \
#    .getOrCreate()

In [5]:
# enable arrow which lets us transfrom a pandas dataframe into a pyspark dataframe
spark.conf.set("spark.sql.execution.arrow.enabled","true")

## First dataset

#### 60,000 images and 10 classes, each image is 28 x 28 or represented by a 794 element array

In [6]:
!wget http://codh.rois.ac.jp/kmnist/dataset/kmnist/train-images-idx3-ubyte.gz?raw=True
!mv train-images-idx3-ubyte.gz?raw=True train-images-idx3-ubyte.gz
!gunzip train-images-idx3-ubyte.gz
!ls -lahr train-images-idx3-ubyte

!wget http://codh.rois.ac.jp/kmnist/dataset/kmnist/train-labels-idx1-ubyte.gz?raw=True
!mv train-labels-idx1-ubyte.gz?raw=True train-labels-idx1-ubyte.gz
!gunzip train-labels-idx1-ubyte.gz
!ls -lahr train-labels-idx1-ubyte

!wget http://codh.rois.ac.jp/kmnist/dataset/kmnist/t10k-images-idx3-ubyte.gz?raw=True
!mv t10k-images-idx3-ubyte.gz?raw=True t10k-images-idx3-ubyte.gz
!gunzip t10k-images-idx3-ubyte.gz
!ls -lahr t10k-images-idx3-ubyte

!wget http://codh.rois.ac.jp/kmnist/dataset/kmnist/t10k-labels-idx1-ubyte.gz?raw=True
!mv t10k-labels-idx1-ubyte.gz?raw=True t10k-labels-idx1-ubyte.gz
!gunzip t10k-labels-idx1-ubyte.gz
!ls -lahr t10k-labels-idx1-ubyte

--2021-06-11 15:56:00--  http://codh.rois.ac.jp/kmnist/dataset/kmnist/train-images-idx3-ubyte.gz?raw=True
Resolving codh.rois.ac.jp (codh.rois.ac.jp)... 136.187.88.58
Connecting to codh.rois.ac.jp (codh.rois.ac.jp)|136.187.88.58|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 18165135 (17M)
Saving to: ‘train-images-idx3-ubyte.gz?raw=True’

train-images-idx3-u 100%[===================>]  17.32M  2.11MB/s    in 8.7s    

2021-06-11 15:56:09 (1.98 MB/s) - ‘train-images-idx3-ubyte.gz?raw=True’ saved [18165135/18165135]

-rw-rw-r-- 1 spark spark 45M Feb  4  2019 train-images-idx3-ubyte
--2021-06-11 15:56:12--  http://codh.rois.ac.jp/kmnist/dataset/kmnist/train-labels-idx1-ubyte.gz?raw=True
Resolving codh.rois.ac.jp (codh.rois.ac.jp)... 136.187.88.58
Connecting to codh.rois.ac.jp (codh.rois.ac.jp)|136.187.88.58|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 29497 (29K)
Saving to: ‘train-labels-idx1-ubyte.gz?raw=True’

train-labels-idx1-u 100

In [7]:
!mkdir kmnistdata
!cp t10k-images-idx3-ubyte kmnistdata/t10k-images-idx3-ubyte
!cp t10k-labels-idx1-ubyte kmnistdata/t10k-labels-idx1-ubyte
!cp train-images-idx3-ubyte kmnistdata/train-images-idx3-ubyte
!cp train-labels-idx1-ubyte kmnistdata/train-labels-idx1-ubyte
!ls -al kmnistdata

data = MNIST('kmnistdata')
train_images, train_labels = data.load_training()
test_images, test_labels = data.load_testing()

train_images = np.array(train_images)
train_labels = np.array(train_labels)
test_images = np.array(test_images)
test_labels = np.array(test_labels)

train_images = train_images / 255
test_images = test_images / 255

total 53908
drwxrwxr-x  2 spark spark     4096 Jun 11 15:56 .
drwxr-xr-x 11 spark  2000     4096 Jun 11 15:56 ..
-rw-rw-r--  1 spark spark  7840016 Jun 11 15:56 t10k-images-idx3-ubyte
-rw-rw-r--  1 spark spark    10008 Jun 11 15:56 t10k-labels-idx1-ubyte
-rw-rw-r--  1 spark spark 47040016 Jun 11 15:56 train-images-idx3-ubyte
-rw-rw-r--  1 spark spark    60008 Jun 11 15:56 train-labels-idx1-ubyte


In [8]:
# model_1 def

model_1 = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation=tf.nn.relu),
    tf.keras.layers.Dense(10, activation=tf.nn.softmax)
])

In [9]:
# model_1 compile
model_1.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [10]:
# model_1 fit

# began by using 5 epochs here but observed that 
# the validation accuracy is increasing even as we
# stopped training, so consider using more epochs here next

model_1.fit(train_images, train_labels, epochs=5, verbose=1, validation_data=(test_images, test_labels))

Epoch 1/5
1875/1875 [==============================] - 9s 5ms/step - loss: 0.4067 - accuracy: 0.8788 - val_loss: 0.5541 - val_accuracy: 0.8262
Epoch 2/5
1875/1875 [==============================] - 8s 4ms/step - loss: 0.1977 - accuracy: 0.9409 - val_loss: 0.4414 - val_accuracy: 0.8742
Epoch 3/5
1875/1875 [==============================] - 8s 4ms/step - loss: 0.1419 - accuracy: 0.9578 - val_loss: 0.4187 - val_accuracy: 0.8849
Epoch 4/5
1875/1875 [==============================] - 8s 4ms/step - loss: 0.1067 - accuracy: 0.9682 - val_loss: 0.4152 - val_accuracy: 0.8887
Epoch 5/5
1875/1875 [==============================] - 8s 4ms/step - loss: 0.0844 - accuracy: 0.9754 - val_loss: 0.3901 - val_accuracy: 0.8972


In [11]:
# model_2 def

# optimizer is adam and loss is sparse_categorical_crossentropy

batch_size = 128
num_classes = 10
epochs = 12

# input image dimensions
img_rows, img_cols = 28, 28

x_train = train_images
x_test = test_images
y_train = train_labels
y_test = test_labels

if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

model_2 = tf.keras.Sequential()
model_2.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model_2.add(Conv2D(64, (3, 3), activation='relu'))
model_2.add(MaxPooling2D(pool_size=(2, 2)))
model_2.add(Dropout(0.25))
model_2.add(tf.keras.layers.Flatten())
model_2.add(tf.keras.layers.Dense(128, activation='relu'))
model_2.add(Dropout(0.5))
model_2.add(tf.keras.layers.Dense(num_classes, activation='softmax'))

# model_2 compile

model_2.compile(optimizer='adam', 
          loss='sparse_categorical_crossentropy',
          metrics=['accuracy'])

In [12]:
# fit model

model_2.fit(x_train, y_train,
         batch_size=batch_size,
         epochs=epochs,
         verbose=1,
         validation_data=(x_test, y_test))

Epoch 1/12
469/469 [==============================] - 214s 456ms/step - loss: 0.4192 - accuracy: 0.8706 - val_loss: 0.3737 - val_accuracy: 0.8884
Epoch 2/12
469/469 [==============================] - 211s 450ms/step - loss: 0.1766 - accuracy: 0.9463 - val_loss: 0.2659 - val_accuracy: 0.9275
Epoch 3/12
469/469 [==============================] - 211s 450ms/step - loss: 0.1296 - accuracy: 0.9604 - val_loss: 0.2291 - val_accuracy: 0.9372
Epoch 4/12
469/469 [==============================] - 212s 452ms/step - loss: 0.1031 - accuracy: 0.9680 - val_loss: 0.2180 - val_accuracy: 0.9427
Epoch 5/12
469/469 [==============================] - 211s 449ms/step - loss: 0.0852 - accuracy: 0.9730 - val_loss: 0.2152 - val_accuracy: 0.9434
Epoch 6/12
469/469 [==============================] - 210s 447ms/step - loss: 0.0748 - accuracy: 0.9762 - val_loss: 0.2055 - val_accuracy: 0.9477
Epoch 7/12
469/469 [==============================] - 211s 450ms/step - loss: 0.0629 - accuracy: 0.9797 - val_loss: 0.1950 -

In [13]:
# change of optimizer and compile
model_2.compile(loss='sparse_categorical_crossentropy',
              optimizer=tf.keras.optimizers.Adadelta(),
              metrics=['accuracy'])

In [14]:
model_2.fit(x_train, y_train,
           batch_size=batch_size,
           epochs=epochs,
           verbose=1,
           validation_data=(x_test, y_test))

Epoch 1/12
469/469 [==============================] - 212s 450ms/step - loss: 0.0368 - accuracy: 0.9878 - val_loss: 0.2387 - val_accuracy: 0.9527
Epoch 2/12
469/469 [==============================] - 211s 451ms/step - loss: 0.0360 - accuracy: 0.9876 - val_loss: 0.2381 - val_accuracy: 0.9529
Epoch 3/12
469/469 [==============================] - 211s 450ms/step - loss: 0.0354 - accuracy: 0.9881 - val_loss: 0.2376 - val_accuracy: 0.9533
Epoch 4/12
469/469 [==============================] - 212s 453ms/step - loss: 0.0363 - accuracy: 0.9873 - val_loss: 0.2371 - val_accuracy: 0.9534
Epoch 5/12
469/469 [==============================] - 211s 451ms/step - loss: 0.0354 - accuracy: 0.9880 - val_loss: 0.2367 - val_accuracy: 0.9537
Epoch 6/12
469/469 [==============================] - 211s 450ms/step - loss: 0.0351 - accuracy: 0.9879 - val_loss: 0.2364 - val_accuracy: 0.9536
Epoch 7/12
469/469 [==============================] - 211s 451ms/step - loss: 0.0361 - accuracy: 0.9878 - val_loss: 0.2359 -

#### This is the first dataset in .npz format

In [15]:
!wget http://codh.rois.ac.jp/kmnist/dataset/kmnist/kmnist-train-imgs.npz?raw=True
!mv kmnist-train-imgs.npz?raw=True kmnist-train-imgs.npz
!ls -lahr kmnist-train-imgs.npz

!wget http://codh.rois.ac.jp/kmnist/dataset/kmnist/kmnist-train-labels.npz?raw=True
!mv kmnist-train-labels.npz?raw=True kmnist-train-labels.npz
!ls -lahr kmnist-train-labels.npz

!wget http://codh.rois.ac.jp/kmnist/dataset/kmnist/kmnist-test-imgs.npz?raw=True
!mv kmnist-test-imgs.npz?raw=True kmnist-test-imgs.npz
!ls -lahr kmnist-test-imgs.npz

!wget http://codh.rois.ac.jp/kmnist/dataset/kmnist/kmnist-test-labels.npz?raw=True
!mv kmnist-test-labels.npz?raw=True kmnist-test-labels.npz
!ls -lahr kmnist-test-labels.npz

--2021-06-11 17:21:46--  http://codh.rois.ac.jp/kmnist/dataset/kmnist/kmnist-train-imgs.npz?raw=True
Resolving codh.rois.ac.jp (codh.rois.ac.jp)... 136.187.88.58
Connecting to codh.rois.ac.jp (codh.rois.ac.jp)|136.187.88.58|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 18384171 (18M)
Saving to: ‘kmnist-train-imgs.npz?raw=True’

kmnist-train-imgs.n 100%[===================>]  17.53M  4.25MB/s    in 4.9s    

2021-06-11 17:21:52 (3.61 MB/s) - ‘kmnist-train-imgs.npz?raw=True’ saved [18384171/18384171]

-rw-rw-r-- 1 spark spark 18M Feb  4  2019 kmnist-train-imgs.npz
--2021-06-11 17:21:54--  http://codh.rois.ac.jp/kmnist/dataset/kmnist/kmnist-train-labels.npz?raw=True
Resolving codh.rois.ac.jp (codh.rois.ac.jp)... 136.187.88.58
Connecting to codh.rois.ac.jp (codh.rois.ac.jp)|136.187.88.58|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 29700 (29K)
Saving to: ‘kmnist-train-labels.npz?raw=True’

kmnist-train-labels 100%[===================>]

### Model:  K Nearest Neighbors

In [16]:
def load(f):
    return np.load(f)['arr_0']

In [17]:
x_train = load('kmnist-train-imgs.npz')
y_train = load('kmnist-train-labels.npz')
x_test = load('kmnist-test-imgs.npz')
y_test = load('kmnist-test-labels.npz')

In [18]:
x_train = x_train.reshape(-1, 784)
x_test = x_test.reshape(-1, 784)

In [19]:
model_3 = KNeighborsClassifier(n_neighbors=4, weights='distance', n_jobs=-1)
print('Fit', model_3)
model_3.fit(x_train, y_train)

Fit KNeighborsClassifier(n_jobs=-1, n_neighbors=4, weights='distance')


KNeighborsClassifier(n_jobs=-1, n_neighbors=4, weights='distance')

In [20]:
print('Evaluate', model_3)
score = model_3.score(x_test, y_test)
print('The accuracy is:', score)

Evaluate KNeighborsClassifier(n_jobs=-1, n_neighbors=4, weights='distance')
The accuracy is: 0.921


### Model: K Nearest Neighbors with Principal Component Analysis dimensionality reduction

In [ ]:
# KNN with dim reduction:  the code is currently broken, skip this for now

pca = PCA(n_components=60, random_state=0)

In [ ]:
x_train = pca.fit_transform(x_train)
x_test = pca.transform

In [ ]:
model_4 = KNeighborsClassifier(n_neighbors=4, weights='distance', n_jobs=-1)
print('Fit', model_4)
model_4.fit(x_train, y_train)

In [ ]:
print('Evaluate', model_4)
score = model_4.score(x_test, y_test)
print('The accuracy is:', score)

## Second dataset

#### 232,365 images, 49 classes, each image is 28 x 28 or represented by a 794 element array

In [6]:
# download the train and test labels and images, 
# which are available onlyh in .npz format 

!wget http://codh.rois.ac.jp/kmnist/dataset/k49/k49-train-imgs.npz?raw=True
!mv k49-train-imgs.npz?raw=True k49-train-imgs.npz
!ls -lahr k49-train-imgs.npz

!wget http://codh.rois.ac.jp/kmnist/dataset/k49/k49-train-labels.npz?raw=True
!mv k49-train-labels.npz?raw=True k49-train-labels.npz
!ls -lahr k49-train-labels.npz

!wget http://codh.rois.ac.jp/kmnist/dataset/k49/k49-test-imgs.npz?raw=True
!mv k49-test-imgs.npz?raw=True k49-test-imgs.npz
!ls -lahr k49-test-imgs.npz

!wget http://codh.rois.ac.jp/kmnist/dataset/k49/k49-test-labels.npz?raw=True
!mv k49-test-labels.npz?raw=True k49-test-labels.npz
!ls -lahr k49-test-labels.npz

--2021-06-11 17:49:10--  http://codh.rois.ac.jp/kmnist/dataset/k49/k49-train-imgs.npz?raw=True
Resolving codh.rois.ac.jp (codh.rois.ac.jp)... 136.187.88.58
Connecting to codh.rois.ac.jp (codh.rois.ac.jp)|136.187.88.58|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 66117696 (63M)
Saving to: ‘k49-train-imgs.npz?raw=True’

k49-train-imgs.npz? 100%[===================>]  63.05M  12.1MB/s    in 6.6s    

2021-06-11 17:49:17 (9.54 MB/s) - ‘k49-train-imgs.npz?raw=True’ saved [66117696/66117696]

-rw-rw-r-- 1 spark spark 64M Feb  4  2019 k49-train-imgs.npz
--2021-06-11 17:49:19--  http://codh.rois.ac.jp/kmnist/dataset/k49/k49-train-labels.npz?raw=True
Resolving codh.rois.ac.jp (codh.rois.ac.jp)... 136.187.88.58
Connecting to codh.rois.ac.jp (codh.rois.ac.jp)|136.187.88.58|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 164485 (161K)
Saving to: ‘k49-train-labels.npz?raw=True’

k49-train-labels.np 100%[===================>] 160.63K   365KB/s    

In [7]:
def load(f):
    return np.load(f)['arr_0']

In [8]:
x_train_k49 = load('k49-train-imgs.npz')
y_train_k49 = load('k49-train-labels.npz')
x_test_k49 = load('k49-test-imgs.npz')
y_test_k49 = load('k49-test-labels.npz')

x_train_k49 = x_train_k49.reshape(-1, 784)
x_test_k49 = x_test_k49.reshape(-1, 784)


# the numpy arrays used in the keras models need to be between 0 and 1
# we keep x_train_k49 and x_test_k49 in the format of between 0 and 255
# for use in the K-nearest Neighbors model below

train_images_k49 = x_train_k49 / 255
test_images_k49 = x_test_k49 / 255

# and the labels can be the same

train_labels_k49 = y_train_k49
test_labels_k49 = y_test_k49

In [9]:
# model_1_k49 def
# we have to change the last layer due to the 49 categories

model_1_k49 = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation=tf.nn.relu),
    tf.keras.layers.Dense(49, activation=tf.nn.softmax)
])

In [10]:
# model_1_k49 compile
model_1_k49.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [11]:
# model_1_k49 fit
# as in the first data set, this model could probably use more epochs
model_1_k49.fit(train_images_k49, train_labels_k49, epochs=5, verbose=1, validation_data=(test_images_k49, test_labels_k49))

Epoch 1/5
7262/7262 [==============================] - 14s 2ms/step - loss: 0.9142 - accuracy: 0.7662 - val_loss: 1.0799 - val_accuracy: 0.7278
Epoch 2/5
7262/7262 [==============================] - 13s 2ms/step - loss: 0.5710 - accuracy: 0.8474 - val_loss: 0.9618 - val_accuracy: 0.7581
Epoch 3/5
7262/7262 [==============================] - 13s 2ms/step - loss: 0.4910 - accuracy: 0.8670 - val_loss: 0.9151 - val_accuracy: 0.7734
Epoch 4/5
7262/7262 [==============================] - 13s 2ms/step - loss: 0.4495 - accuracy: 0.8786 - val_loss: 0.9089 - val_accuracy: 0.7800
Epoch 5/5
7262/7262 [==============================] - 13s 2ms/step - loss: 0.4226 - accuracy: 0.8843 - val_loss: 0.9129 - val_accuracy: 0.7817


In [12]:
# model_2_k49 def

batch_size = 128
num_classes = 49       #  here we specify the number of classes as 49 instead of 10
epochs = 12

# input image dimensions
img_rows, img_cols = 28, 28

x_train = train_images_k49
x_test = test_images_k49
y_train = train_labels_k49
y_test = test_labels_k49

if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

model_2_k49 = tf.keras.Sequential()
model_2_k49.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model_2_k49.add(Conv2D(64, (3, 3), activation='relu'))
model_2_k49.add(MaxPooling2D(pool_size=(2, 2)))
model_2_k49.add(Dropout(0.25))
model_2_k49.add(tf.keras.layers.Flatten())
model_2_k49.add(tf.keras.layers.Dense(128, activation='relu'))
model_2_k49.add(Dropout(0.5))
model_2_k49.add(tf.keras.layers.Dense(num_classes, activation='softmax'))

# model_2 compile

model_2_k49.compile(optimizer='adam', 
          loss='sparse_categorical_crossentropy',
          metrics=['accuracy'])

In [13]:
# fit model2_k49

model_2_k49.fit(x_train, y_train,
         batch_size=batch_size,
         epochs=epochs,
         verbose=1,
         validation_data=(x_test, y_test))

Epoch 1/12
1816/1816 [==============================] - 342s 188ms/step - loss: 0.9383 - accuracy: 0.7530 - val_loss: 0.6622 - val_accuracy: 0.8259
Epoch 2/12
1816/1816 [==============================] - 340s 187ms/step - loss: 0.5146 - accuracy: 0.8584 - val_loss: 0.5186 - val_accuracy: 0.8644
Epoch 3/12
1816/1816 [==============================] - 338s 186ms/step - loss: 0.4178 - accuracy: 0.8831 - val_loss: 0.4378 - val_accuracy: 0.8841
Epoch 4/12
1816/1816 [==============================] - 338s 186ms/step - loss: 0.3619 - accuracy: 0.8981 - val_loss: 0.3942 - val_accuracy: 0.8978
Epoch 5/12
1816/1816 [==============================] - 336s 185ms/step - loss: 0.3248 - accuracy: 0.9068 - val_loss: 0.3849 - val_accuracy: 0.9000
Epoch 6/12
1816/1816 [==============================] - 336s 185ms/step - loss: 0.2997 - accuracy: 0.9139 - val_loss: 0.3704 - val_accuracy: 0.9067
Epoch 7/12
1816/1816 [==============================] - 335s 185ms/step - loss: 0.2788 - accuracy: 0.9189 - val_

In [14]:
# change of optimizer and compile model_2_k49

model_2_k49.compile(loss='sparse_categorical_crossentropy',
              optimizer=tf.keras.optimizers.Adadelta(),
              metrics=['accuracy'])

In [15]:
# fit model_2_k49 after change of optmizer

model_2_k49.fit(x_train, y_train,
           batch_size=batch_size,
           epochs=epochs,
           verbose=1,
           validation_data=(x_test, y_test))

Epoch 1/12
1816/1816 [==============================] - 342s 188ms/step - loss: 0.1990 - accuracy: 0.9387 - val_loss: 0.3261 - val_accuracy: 0.9198
Epoch 2/12
1816/1816 [==============================] - 341s 188ms/step - loss: 0.1954 - accuracy: 0.9403 - val_loss: 0.3262 - val_accuracy: 0.9198
Epoch 3/12
1816/1816 [==============================] - 340s 188ms/step - loss: 0.1942 - accuracy: 0.9404 - val_loss: 0.3263 - val_accuracy: 0.9200
Epoch 4/12
1816/1816 [==============================] - 342s 188ms/step - loss: 0.1948 - accuracy: 0.9405 - val_loss: 0.3264 - val_accuracy: 0.9202
Epoch 5/12
1816/1816 [==============================] - 340s 187ms/step - loss: 0.1921 - accuracy: 0.9413 - val_loss: 0.3267 - val_accuracy: 0.9203
Epoch 6/12
1816/1816 [==============================] - 340s 187ms/step - loss: 0.1917 - accuracy: 0.9410 - val_loss: 0.3269 - val_accuracy: 0.9201
Epoch 7/12
1816/1816 [==============================] - 344s 189ms/step - loss: 0.1923 - accuracy: 0.9409 - val_

### Model:  K Nearest Neighbors - k49

In [16]:
# recall that the two numpy image arrays used below, namely x_train_k49 and x_test_k49
# are arrays of integers, which works fine for KNN

In [17]:
model_3_k49 = KNeighborsClassifier(n_neighbors=4, weights='distance', n_jobs=-1)
print('Fit', model_3_k49)
model_3_k49.fit(x_train_k49, y_train_k49)

Fit KNeighborsClassifier(n_jobs=-1, n_neighbors=4, weights='distance')


KNeighborsClassifier(n_jobs=-1, n_neighbors=4, weights='distance')

In [ ]:
print('Evaluate', model_3_k49)
score = model_3_k49.score(x_test_k49, y_test_k49)
print('The accuracy is:', score)

Evaluate KNeighborsClassifier(n_jobs=-1, n_neighbors=4, weights='distance')
